开始之前的准备工作：

1.使用conda命令创建新的虚拟环境（pytorch）

In [ ]:
conda create --name env_name python==version

2.在Anaconda Prompt进入指定虚拟环境中使用pip命令安装所需库

In [ ]:
pip install torch torchvision tensorflow numpy matplotlib keras(==version)

3.下载py第三方库请把镜像网站焊在头上！

In [ ]:
pip install pyinstaller -i https://pypi.douban.com/simple --trusted-host pypi.douban.com

4.安装完后直接输入以下命令，切记要从指定虚拟环境中进入idle

In [ ]:
python -m idlelib.idle

注意事项：

1.解决tensorflow2.15问题在开头加上以下代码

In [ ]:
set TF_ENABLE_ONEDNN_OPTS=0

2.一定要以管理员身份升级库

In [ ]:
pip install --user --upgrade --ignore-installed 指定库==指定版本

3.千万要注意各个库版本之间的兼容问题！不然会出大问题！

4.下载指定版本库操作

In [ ]:
pip uninstall keras
pip install  keras==2.15.0 最好加上内地镜像源网站

5.常见内地镜像源

In [ ]:
#豆瓣：https://pypi.douban.com/simple
#清华：https://pypi.tuna.tsinghua.edu.cn/simple
#阿里云：https://mirrors.aliyun.com/pypi/simple
pip --default-timeout=1000 install -XXX -i https://pypi.douban.com/simple
#将安装时间限制调整也可以提高第三方库下载效率

6.解决mnist无法从外网Google下载问题

把mnist.npz文件下载保存到指定文件夹
将mnist.py文件中的
origin_folder = "https:......"改成
"origin_folder = "C:/Users/22472/.keras/datasets/"指定文件夹目录注意看源代码
一定要双引号和/！

自己写一段源代码load_data()

In [ ]:
def load_data():
    """Loads the MNIST dataset.
    # Arguments
        path: path where to cache the dataset locally
            (relative to ~/.keras/datasets).
    # Returns
        Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`.
        """
    path='D:\\anaconda\\envs\\pytorch\\Lib\site-packages\\tensorflow\\python\\keras\\datasets\\minst.npz'
    f = np.load(path)
    x_train, y_train = f['x_train'], f['y_train']
    x_test, y_test = f['x_test'], f['y_test']
    f.close()
return (x_train, y_train), (x_test, y_test)
#记得把下面的imoprt mnist和代码里的mnist去掉

7.循环引用（circular import）

cannot import name 'to_str' from partially initialized module 'urllib3.util' (most likely due to a circular import) 
(D:\anaconda\envs\scrapy\Lib\site-packages\urllib3\util\__init__.py)

原因：model和py或package文件重名

解决：需要绝对引用

例子：from urllib3.util.util import to_str

实战流程：果

1.获取数据，并将数据处理成合适的格式

2.按照自己的设计搭建神经网络

3.设定合适的参数训练神经网络

4.在测试集上评价训练效果

代码演示：

In [ ]:
import torch  
import torch.nn as nn  
import torch.optim as optim  
from torchvision import datasets, transforms  
from torch.utils.data import DataLoader  
from torchvision.models import vgg16  
import numpy as np  
import matplotlib.pyplot as plt  

In [ ]:
# 步骤1：导入并转换图片数据  
transform = transforms.Compose([  
    transforms.ToTensor(),  
    transforms.Normalize((0.5,), (0.5,))  # 归一化图片数据  
])  
  
trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)  
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)  
  
testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)  
testloader = DataLoader(testset, batch_size=64, shuffle=True)  
  
# 步骤2：定义数据迭代器  
# 已在步骤1中定义  

In [ ]:
# 步骤3：定义网络模型  
class Net(nn.Module):  
    def __init__(self):  
        super(Net, self).__init__()  
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)  
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)  
        self.conv2_drop = nn.Dropout2d()  
        self.fc1 = nn.Linear(320, 50)  
        self.fc2 = nn.Linear(50, 10)  
  
    def forward(self, x):  
        x = F.relu(F.max_pool2d(self.conv1(x), 2))  
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))  
        x = x.view(-1, 320)  
        x = F.relu(self.fc1(x))  
        x = F.dropout(x, training=self.training)  
        x = self.fc2(x)  
        return F.log_softmax(x, dim=1)  
  
net = Net()  

In [ ]:
# 步骤4：利用训练集数据训练模型  
criterion = nn.NLLLoss()  
optimizer = optim.SGD(net.parameters(), lr=0.003)  
  
for epoch in range(2):  # loop over the dataset multiple times  
    running_loss = 0.0  
    for i, data in enumerate(trainloader, 0):  
        # get the inputs; data is a list of [inputs, labels]  
        inputs, labels = data  
  
        # zero the parameter gradients  
        optimizer.zero_grad()  
  
        # forward + backward + optimize  
        outputs = net(inputs)  
        loss = criterion(outputs, labels)  
        loss.backward()  
        optimizer.step()  
  
        # print statistics  
        running_loss += loss.item()  
        if i % 2000 == 1999:    # print every 2000 mini-batches  
            print('[%d, %5d] loss: %.3f' %  
                  (epoch + 1, i + 1, running_loss / 2000))  
            running_loss = 0.0  
  
print('Finished Training')  

In [ ]:
# 步骤5：利用分出来的测试集数据验证模型性能  
correct_count, all_count = 0, 0  
with torch.no_grad():  
    for images, labels in testloader:  
        outputs = net(images)  
        _, predicted = torch.max(outputs.data, 1)  
        total = labels.size(0)  
        correct_count += (predicted == labels).sum().item()  
        all_count += total  
  
print('Accuracy of the network on the 10000 test images: %d %%' % (  
    100 * correct_count / all_count))  

In [ ]:
# 步骤6：训练完成后挑几个图片看看模型的预测结果  
images, labels = next(iter(testloader))